In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Replace 'cctv_data.csv' with the actual path to your CSV file
cctv_data = pd.read_csv('/content/drive/MyDrive/Ketch-Up/빅데이터 활용 미래 사회문제 해결 아이디어 해커톤/대전광역시 유성구_CCTV 설치_위도경도포함.csv')


In [3]:
from geopy.geocoders import Nominatim
from geopy.geocoders import options

# Nominatim 클래스를 사용하기 전에 options에서 format_string을 설정
options.default_format_string = "%s, 대한민국"
geolocator = Nominatim(user_agent="my_geocoding_app")

# 사용자로부터 도로명 주소 입력 받기
user_address = input('도로명주소를 입력하세요: ')

# 도로명 주소를 좌표로 변환
location = geolocator.geocode(user_address)

# 결과 출력
if location:
    print(f"위도: {location.latitude}, 경도: {location.longitude}")
else:
    print("주소를 찾을 수 없습니다.")


도로명주소를 입력하세요: 유성구 대학로 211 
위도: 36.3616324, 경도: 127.3445103


In [4]:
user_latitude=location.latitude
user_longitude=location.longitude

In [5]:
from geopy.distance import geodesic

# Set the radius (1km in this case)
# search_radius = 1.0

# Get user input for the radius
search_radius = float(input('반경 몇 km 내의 CCTV를 검색할까요?: '))

# Create a DataFrame to store CCTV locations within the specified radius
nearby_cctv = pd.DataFrame(columns=cctv_data.columns)

for index, cctv_row in cctv_data.iterrows():
    cctv_location = (cctv_row['Latitude'], cctv_row['Longitude'])
    distance = geodesic((user_latitude, user_longitude), cctv_location).kilometers

    if distance <= search_radius:
        nearby_cctv = nearby_cctv.append(cctv_row, ignore_index=True)


반경 몇 km 내의 CCTV를 검색할까요?: 0.2


<ipython-input-5-5abddf8c2c23>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nearby_cctv = nearby_cctv.append(cctv_row, ignore_index=True)


In [6]:
# 반경 n km 내 cctv df으로 출력하기
# 행렬의 행의 개수 확인
num_rows = nearby_cctv.shape[0]
# Print the result with the user input
print(f"반경 {search_radius} km 내의 {num_rows}개의 CCTV를 찾았습니다")

# 특정 열들만 선택
selected_columns = nearby_cctv[['관리번호','유형','카메라대수','설치위치']]

# 인덱스를 숨기고 출력
selected_columns.style.hide_index()


반경 0.2 km 내의 3개의 CCTV를 찾았습니다


<ipython-input-6-91c67af646e9>:11: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  selected_columns.style.hide_index()


관리번호,유형,카메라대수,설치위치
14-314,방범용,1.000000,대전광역시 유성구 궁동 451-7대학로 81번길 32-11
16-483,방범용,2.000000,대전광역시 유성구 봉명동 611-9 (조대포식당) 주변문화원로 87
16-489,방범용,1.000000,대전광역시 유성구 궁동 486-4 (GS25) 주변 대학로76번안길 31


In [7]:
# 반경 n km 내 cctv 출력하기

print("CCTV locations within 1km of the user-specified address:")
print(nearby_cctv[['관리번호','유형','카메라대수','설치위치']])


CCTV locations within 1km of the user-specified address:
     관리번호   유형  카메라대수                                               설치위치
0  14-314  방범용    1.0                   대전광역시 유성구 궁동 451-7대학로 81번길 32-11
1  16-483  방범용    2.0              대전광역시 유성구 봉명동 611-9 (조대포식당) 주변문화원로 87
2  16-489  방범용    1.0  대전광역시 유성구 궁동 486-4 (GS25) 주변                  ...


# 주소 입력 시 주변 CCTV pin 출력

In [8]:
import folium
from folium import IFrame

# Function to create HTML content for the popup
def create_popup_content(cctv_row):
    content = f"<strong>설치위치:</strong> {cctv_row['설치위치']}<br>"
    content += f"<strong>카메라대수:</strong> {int(cctv_row['카메라대수'])}대"
    return content

# 지도 생성
m = folium.Map(location=[user_latitude, user_longitude], zoom_start=15)

# 사용자 위치에 마커 표시
folium.Marker([user_latitude, user_longitude], popup="User Address").add_to(m)

# 인근 CCTV에 대한 마커 생성 및 카메라 대수 표시
for index, cctv_row in nearby_cctv.iterrows():
    cctv_location = (cctv_row['Latitude'], cctv_row['Longitude'])

    # Create HTML content for the popup
    popup_content = create_popup_content(cctv_row)

    # Create an IFrame with the HTML content
    iframe = IFrame(html=popup_content, width=300, height=80)

    # Add marker to the map with the IFrame as the popup
    folium.Marker(
        cctv_location,
        popup=folium.Popup(iframe, max_width=300),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# 지도를 HTML 파일로 저장하거나 출력
#m.save('map_with_cctv.html')
m


In [ ]:
#지름 지정해서 pin 찍힌 곳 반경으로 원 생성
#안쓸래
# folium.CircleMarker(location=[user_latitude, user_longitude],
#                     color='black',
#                     #radius = 700000,
#                     tooltip='인근 CCTV').add_to(m)
# m

# marker cluster 형성

In [9]:
from folium.plugins import MarkerCluster
import folium
from folium import Marker, Popup, IFrame



mc = MarkerCluster()

In [10]:
# Iterate through CCTV data and add markers within the specified radius
for _, row in cctv_data.iterrows():
    # Check if the distance is within the specified radius
    distance = geodesic((user_latitude, user_longitude), (row['Latitude'], row['Longitude'])).kilometers
    if distance <= search_radius:
        mc.add_child(
            Marker(location=[row['Latitude'], row['Longitude']]
                   #,popup=f"카메라대수: {row['카메라대수']}"
            )
        )

# Add the MarkerCluster to the map
m.add_child(mc)

# Display the map
m

NameError: name 'Marker' is not defined